In [39]:
from aubio import onset, tempo, source
from numpy import hstack, zeros
import matplotlib.pyplot as plt
%matplotlib inline  

In [50]:
#libvlc, ffmpeg, gstreamer
win_s = 512 #fft size
hop_s = win_s//2 #hop size
filename = 'Feel It Still.mp3'
samplerate = 44100
s = source(filename, samplerate, hop_s)
o = onset('default', win_s, hop_s, samplerate)
t = tempo("default", win_s, hop_s, samplerate)
delay = 4. * hop_s
total_frames = 0
onsets = []
onsets_desc = []
onsets_tdesc = []
beats = []

while True:
    samples, read = s()
    if o(samples):
        onsets.append(o.get_last())
        onsets_desc.append(o.get_descriptor())
        onsets_tdesc.append(o.get_thresholded_descriptor())
    if t(samples):
        this_beat = int(total_frames - delay + t(samples)[0] * hop_s)
        beats.append(this_beat)
    total_frames += read
    if read < hop_s: 
        break

In [35]:
diff_onsets = [onsets[i + 1] - onsets[i] for i in range(len(onsets) - 1)]

In [47]:
# import time
# for diff in diff_onsets:
#     time.sleep(diff)
#     print desc

In [52]:
len(beats)

364

In [56]:
mean(60./diff(beats))

0.0032247435446994131

In [66]:
from PIL import Image
import requests
import tempfile
import cv2

def load_gif_url(url):
    with tempfile.NamedTemporaryFile(suffix=".gif") as f:
        f.write(requests.get(url).content)
        f.flush()
        img = Image.open(f.name)

    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        img.save(f.name)
        f.flush()
        src = cv2.imread(f.name)

    assert src is not None and len(src), "Empty"

    return src

In [68]:
src = load_gif_url('https://media.giphy.com/media/v6aOjy0Qo1fIA/giphy.gif')

In [ ]:
import cv2
import numpy as np

# original image
# -1 loads as-is so if it will be 3 or 4 channel as the original
image = cv2.imread('image.png', -1)
# mask defaulting to black for 3-channel and transparent for 4-channel
# (of course replace corners with yours)
mask = np.zeros(image.shape, dtype=np.uint8)
roi_corners = np.array([[(10,10), (300,300), (10,300)]], dtype=np.int32)
# fill the ROI so it doesn't get wiped out when the mask is applied
channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
ignore_mask_color = (255,)*channel_count
cv2.fillPoly(mask, roi_corners, ignore_mask_color)
# from Masterfool: use cv2.fillConvexPoly if you know it's convex

# apply the mask
masked_image = cv2.bitwise_and(image, mask)

# save the result
cv2.imwrite('image_masked.png', masked_image)